<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/ANATOMY_AGENTIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Flask and pyngrok
!pip install Flask pyngrok -q
!pip install colab-env -q

In [ ]:
import os
import socket
import threading
import time
import json

from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import google.generativeai as genai

# --- NO PROBLEMATIC WERKZEUG.LOCAL.LocalProxy MONKEY-PATCH IS INCLUDED HERE ---
# This section is deliberately empty to avoid the RecursionError.
# No logging filters are applied to avoid interfering with Flask's default behavior.

# --- 1. Flask Application Definition ---
app = Flask(__name__)

# Optional: Set a secret key if you plan to use sessions (highly recommended for production apps)
# app.secret_key = os.getenv('FLASK_SECRET_KEY', 'your_super_secret_key_here')

# 2. Configuration for Agent
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-1.5-flash"

# 3. Google Colab / Gemini API Imports and Configuration
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if not GOOGLE_API_KEY:
    print("WARNING: GEMINI API Key not found. Gemini API calls may fail.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)


# 4. Define the HTML content as a Python string.
HTML_CONTENT = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Gemini 2.0 Flight Planner AI</title>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Roboto', sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f0f2f5;
            color: #333;
            display: flex;
            justify-content: center;
            align-items: flex-start;
            min-height: 100vh;
        }
        .container {
            background: #ffffff;
            padding: 30px;
            border-radius: 12px;
            box-shadow: 0 6px 20px rgba(0,0,0,0.1);
            max-width: 900px;
            width: 100%;
            margin-top: 50px;
        }
        h1 {
            color: #1a73e8;
            text-align: center;
            margin-bottom: 25px;
            font-weight: 700;
        }
        .input-group {
            margin-bottom: 15px;
        }
        .input-group label {
            display: block;
            margin-bottom: 8px;
            font-weight: 500;
            color: #555;
        }
        .input-group input[type="text"],
        .input-group input[type="date"],
        .input-group input[type="number"],
        .input-group select,
        .input-group textarea {
            width: calc(100% - 22px); /* Account for padding and border */
            padding: 12px;
            border: 1px solid #c2c2c2;
            border-radius: 6px;
            font-size: 16px;
            box-sizing: border-box;
            transition: border-color 0.3s;
        }
        .input-group input:focus,
        .input-group select:focus,
        .input-group textarea:focus {
            border-color: #1a73e8;
            outline: none;
        }
        .grid-container {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
            margin-bottom: 20px;
        }
        .full-width {
            grid-column: span 2;
        }
        button {
            padding: 12px 25px;
            background-color: #1a73e8;
            color: white;
            border: none;
            border-radius: 6px;
            cursor: pointer;
            font-size: 18px;
            font-weight: 600;
            width: 100%;
            transition: background-color 0.3s, transform 0.2s;
            margin-top: 20px;
        }
        button:hover {
            background-color: #155bb5;
            transform: translateY(-2px);
        }
        #responseArea {
            margin-top: 30px;
            padding: 20px;
            background: #eaf3ff;
            border-radius: 8px;
            min-height: 100px;
            border: 1px solid #d0e0ff;
            box-shadow: inset 0 1px 3px rgba(0,0,0,0.05);
            overflow-x: auto; /* For potential long flight details */
        }
        #responseArea h2 {
            color: #1a73e8;
            margin-top: 0;
            margin-bottom: 15px;
            font-weight: 600;
        }
        #responseArea pre {
            white-space: pre-wrap;
            word-wrap: break-word;
            font-family: 'Roboto Mono', monospace;
            font-size: 14px;
            line-height: 1.5;
            color: #333;
        }
        .loading {
            color: #888;
            text-align: center;
            font-style: italic;
        }
        .error {
            color: #d93025;
            font-weight: 500;
            text-align: center;
        }
        /* Styling for structured flight plan */
        .flight-segment {
            border: 1px solid #cce0ff;
            border-radius: 8px;
            padding: 15px;
            margin-bottom: 15px;
            background-color: #ffffff;
            box-shadow: 0 2px 5px rgba(0,0,0,0.05);
        }
        .flight-segment h3 {
            color: #0d47a1;
            margin-top: 0;
            margin-bottom: 10px;
            font-size: 1.2em;
        }
        .flight-segment p {
            margin: 5px 0;
            font-size: 0.95em;
        }
        .flight-segment strong {
            color: #000;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>✈️ Gemini 2.0 Flight Planner AI ✈️</h1>
        <p>Input your flight details to get an optimal plan:</p>

        <div class="grid-container">
            <div class="input-group">
                <label for="originInput">Origin (IATA Code):</label>
                <input type="text" id="originInput" placeholder="e.g., YUL">
            </div>
            <div class="input-group">
                <label for="destinationInput">Destination (IATA Code):</label>
                <input type="text" id="destinationInput" placeholder="e.g., MAD">
            </div>
            <div class="input-group">
                <label for="departureDateInput">Departure Date:</label>
                <input type="date" id="departureDateInput">
            </div>
            <div class="input-group">
                <label for="returnDateInput">Return Date (Optional):</label>
                <input type="date" id="returnDateInput">
            </div>
            <div class="input-group">
                <label for="passengersInput">Passengers:</label>
                <input type="number" id="passengersInput" value="1" min="1">
            </div>
            <div class="input-group">
                <label for="classInput">Travel Class:</label>
                <select id="classInput">
                    <option value="economy">Economy</option>
                    <option value="business">Business</option>
                    <option value="first">First</option>
                </select>
            </div>
            <div class="input-group full-width">
                <label for="preferencesInput">Additional Preferences (Optional):</label>
                <textarea id="preferencesInput" rows="3" placeholder="e.g., shortest travel time, specific airline, avoid red-eye flights"></textarea>
            </div>
        </div>

        <button onclick="sendPrompt()">Plan Flight</button>
        <div id="responseArea"></div>
    </div>

    <script>
        async function sendPrompt() {
            const origin = document.getElementById('originInput').value.trim().toUpperCase();
            const destination = document.getElementById('destinationInput').value.trim().toUpperCase();
            const departureDate = document.getElementById('departureDateInput').value;
            const returnDate = document.getElementById('returnDateInput').value;
            const passengers = document.getElementById('passengersInput').value;
            const travelClass = document.getElementById('classInput').value;
            const preferences = document.getElementById('preferencesInput').value.trim();

            const responseArea = document.getElementById('responseArea');

            // Basic validation
            if (!origin || !destination || !departureDate) {
                responseArea.innerHTML = '<p class="error">Please fill in Origin, Destination, and Departure Date.</p>';
                return;
            }

            responseArea.innerHTML = '<p class="loading">Consulting air traffic control and Gemini AI...</p>';

            let promptText = `As an AI flight planning agent, provide an optimal flight plan for a flight from ${origin} to ${destination} on ${departureDate}.`;
            if (returnDate) {
                promptText += ` This is a round trip with a return on ${returnDate}.`;
            }
            promptText += ` Number of passengers: ${passengers}. Travel class: ${travelClass}.`;
            if (preferences) {
                promptText += ` Additional preferences: ${preferences}.`;
            }
            // Requesting structured JSON output from Gemini, emphasizing strict JSON within markdown
            promptText += ` IMPORTANT: Provide the flight plan ONLY as a JSON object, wrapped in ```json and ```. Do NOT include any other text, greetings, or explanations outside these JSON delimiters. The JSON object must have a 'summary' string, and an array of 'flights'. Each flight object should include 'flight_number', 'airline', 'departure_airport_iata', 'arrival_airport_iata', 'departure_time', 'arrival_time', 'duration', and 'layover_info' (if applicable). Example JSON: {"summary": "Flight plan summary...", "flights": [{"flight_number": "UA123", "airline": "United", "departure_airport_iata": "YUL", "arrival_airport_iata": "ORD", "departure_time": "2025-07-15T08:00:00", "arrival_time": "2025-07-15T09:30:00", "duration": "2h30m", "layover_info": null}, {"flight_number": "UA456", "airline": "United", "departure_airport_iata": "ORD", "arrival_airport_iata": "MAD", "departure_time": "2025-07-15T11:00:00", "arrival_time": "2025-07-16T05:00:00", "duration": "7h00m", "layover_info": "ORD (1h30m)"}]}`;


            try {
                const response = await fetch('/api/reason', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ prompt: promptText })
                });

                const data = await response.json();

                if (response.ok) {
                    const aiResponse = data.response;
                    responseArea.innerHTML = '<h2>Generated Flight Plan:</h2>';

                    // --- Robust JSON Extraction and Parsing ---
                    let jsonString = aiResponse;
                    const jsonMatch = aiResponse.match(/```json\s*([\s\S]*?)\s*```/s); // Extract content within ```json ... ```
                    if (jsonMatch && jsonMatch[1]) {
                        jsonString = jsonMatch[1];
                    } else {
                        // Fallback: Try to find the first and last curly braces if no markdown block
                        const firstCurly = jsonString.indexOf('{');
                        const lastCurly = jsonString.lastIndexOf('}');
                        if (firstCurly !== -1 && lastCurly !== -1 && lastCurly > firstCurly) {
                            jsonString = jsonString.substring(firstCurly, lastCurly + 1);
                        } else {
                            // If no JSON-like structure is found, display raw text and exit
                            responseArea.innerHTML += '<p class="error">AI did not return a recognizable JSON structure. Displaying raw response:</p>';
                            responseArea.innerHTML += `<pre>${aiResponse}</pre>`;
                            console.error("AI response did not contain a parseable JSON string:", aiResponse);
                            return; // Exit here as we can't parse
                        }
                    }

                    try {
                        const flightPlan = JSON.parse(jsonString);
                        let htmlOutput = `<p>${flightPlan.summary}</p>`;
                        if (flightPlan.flights && Array.isArray(flightPlan.flights)) {
                            flightPlan.flights.forEach(flight => {
                                htmlOutput += `<div class="flight-segment">
                                    <h3>Flight ${flight.flight_number} (${flight.airline})</h3>
                                    <p><strong>Route:</strong> ${flight.departure_airport_iata} to ${flight.arrival_airport_iata}</p>
                                    <p><strong>Depart:</strong> ${flight.departure_time} | <strong>Arrive:</strong> ${flight.arrival_time}</p>
                                    <p><strong>Duration:</strong> ${flight.duration}</p>`;
                                if (flight.layover_info) {
                                    htmlOutput += `<p><strong>Layover:</strong> ${flight.layover_info}</p>`;
                                }
                                htmlOutput += `</div>`;
                            });
                        } else {
                            htmlOutput += `<p>AI provided a summary but no structured flight details (or 'flights' array was empty/missing).</p><pre>${aiResponse}</pre>`;
                        }
                        responseArea.innerHTML += htmlOutput;

                    } catch (e) {
                        # If AI doesn't return perfect JSON, display raw text
                        responseArea.innerHTML += '<p class="error">AI returned text that was not perfectly valid JSON even after extraction. Displaying raw response:</p>';
                        responseArea.innerHTML += `<pre>${aiResponse}</pre>`;
                        console.error("Failed to parse AI response as JSON after extraction:", e, "Raw string:", jsonString);
                    }

                } else {
                    responseArea.innerHTML = '<p class="error">Error from AI backend: ' + data.error + '</p>';
                    console.error('Backend error:', data.error);
                }
            } catch (error) {
                console.error('Fetch error:', error);
                responseArea.innerHTML = '<p class="error">Network error or failed to connect to the server. Check console for details.</p>';
            }
        }
    </script>
</body>
</html>
"""

# 5. Flask Routes
@app.route('/')
def index():
    """Serves the main HTML content."""
    return render_template_string(HTML_CONTENT)

@app.route('/api/reason', methods=['POST'])
def reason():
    """
    Handles reasoning requests by sending the prompt to the Gemini 1.5 Flash model
    and returning its generated response.
    """
    if not GOOGLE_API_KEY:
        return jsonify({"error": "Gemini API key not configured on the backend."}), 500

    data = request.get_json()
    prompt = data.get('prompt', 'Generate a general AI response.')

    try:
        model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        response = model.generate_content(prompt)
        generated_text = ""

        # Ensure the model's response is converted to text
        if response.parts:
            generated_text = response.text
        elif response.candidates and len(response.candidates) > 0 and response.candidates[0].content:
            # Access text content from Candidate.content.parts if it exists
            # Use join to handle potential multiple parts gracefully
            generated_text = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
        else:
            generated_text = "No text content found in Gemini response."

        return jsonify({"response": generated_text})

    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        if "safety" in str(e).lower() or "blocked" in str(e).lower():
            return jsonify({"error": "AI response blocked due to safety concerns or content policy. Please try a different prompt."}), 500
        else:
            return jsonify({"error": f"Failed to get response from AI: {e}"}), 500

# 6. Helper function to find a free port
def find_free_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('0.0.0.0', 0)) # Bind to port 0 to let OS find a free port
    port = sock.getsockname()[1]
    sock.close()
    return port

# --- 7. Application Startup ---
if __name__ == '__main__':
    # Ensure any previous ngrok tunnels are killed before starting a new one
    ngrok.kill()

    try:
        # Get ngrok authtoken from environment variable
        authtoken = None
        try:
            from google.colab import userdata
            authtoken = userdata.get('NGROK_TOKEN')
        except (ImportError, KeyError):
            authtoken = os.getenv('NGROK_TOKEN')

        if not authtoken:
            raise ValueError("NGROK_TOKEN environment variable not set. Please set it in Colab's Secrets (key icon) with 'Notebook access' ON, or as an environment variable.")

        ngrok.set_auth_token(authtoken)
        print("ngrok authtoken loaded.")

        # Find a free port for Flask to use
        flask_port = find_free_port()
        print(f"Flask will attempt to run on dynamically assigned port: {flask_port}")

        # Establish the ngrok tunnel to the dynamically assigned Flask port
        print(f"Attempting to establish ngrok tunnel to port {flask_port}...")
        public_url = ngrok.connect(flask_port).public_url

        print(f"ngrok tunnel started at: {public_url}")
        print("Click the link above to view the demo!")

        # Run Flask app in the main thread (blocking call)
        # Using debug=True to ensure proper error reporting and functionality.
        print(f"Starting Flask app on port {flask_port} with debug=True...")
        app.run(host="0.0.0.0", port=flask_port, use_reloader=False, debug=True)

    except Exception as e:
        print(f"An error occurred during setup or execution: {e}")
        print("Please ensure your ngrok authtoken (NGROK_TOKEN) and Gemini API key (GEMINI) are correctly set in Colab Secrets, and that port is available.")
    finally:
        ngrok.kill()
        print("ngrok tunnel disconnected.")